In [1]:
import mediapipe as mp
import cv2
# import mediapipe as mp
import numpy as np
import pandas as pd
import os
import csv
import time
import math


In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles
# help(mp.solutions.drawing_utils)

In [3]:
# def image_resize(image):
# #     screen_res = 1760, 990
#     screen_res = 1280, 720
#     scale_width = screen_res[0] / image.shape[1]
#     scale_height = screen_res[1] / image.shape[0]
#     scale = min(scale_width, scale_height)
#     #resized window width and height
#     window_width = int(image.shape[1] * scale)
#     window_height = int(image.shape[0] * scale)
#     resized_img=cv2.resize(image, (window_width,window_height))
#     return resized_img

def find_mid(a,b):
    a = np.array(a) # First
    b = np.array(b) # Second
    return (a+b)/2

def distance(a,b):
    a = np.array(a) # First
    b = np.array(b) # Second
    return ( ((a[0]-b[0])**2) + ((a[1]-b[1])**2) + ((a[2]-b[2])**2) ) **0.5

def magnitude(a):
    a = np.array(a)
    return ((a[0]**2)+(a[1]**2)+(a[2]**2))**0.5

def get_line(a,b):
    a = np.array(a) # First
    b = np.array(b) # Second
    return a-b
         
def get_joints(landmarks):
#     joints = head(mid point of eyes), neck, left shoulder, right shoulder, left elbow, right elbow, left wrist, right wrist, left hip, mid hip, right hip, left knee, right knee, left ankle, right ankle
    joints=[]
    # print("landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x")
    # print(landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].x)
    joints.append(find_mid([landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].z],
                            [landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].z])
                  )
    joints.append(find_mid([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z])
                 )
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]))
    joints.append(find_mid([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z],
                           [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z])
                 )
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]))
    joints.append(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]))
    
    return joints

def get_body_lines(joints):
    body_lines=[]
    body_lines.append(get_line(joints[0],joints[1])) # head to neck
    body_lines.append(get_line(joints[1],joints[9])) # neck to hip
    body_lines.append(get_line(joints[8],joints[9])) # left_hip to mid_hip
    body_lines.append(get_line(joints[10],joints[9])) # right_hip to mid_hip
    body_lines.append(get_line(joints[8],joints[11])) # left_hip to left_knee
    body_lines.append(get_line(joints[10],joints[12])) # right_hip to right_knee
    body_lines.append(get_line(joints[11],joints[13])) # left_knee to left_ankle
    body_lines.append(get_line(joints[12],joints[14])) # right_knee to right_ankle
    body_lines.append(get_line(joints[2],joints[1])) # left_shoulder to neck
    body_lines.append(get_line(joints[3],joints[1])) # right_shoulder to neck
    body_lines.append(get_line(joints[2],joints[4])) # left_shoulder to left_elbow
    body_lines.append(get_line(joints[4],joints[6])) # left_elbow to left_wrist
    body_lines.append(get_line(joints[3],joints[5])) # right_shoulder to right_elbow
    body_lines.append(get_line(joints[5],joints[7])) # right_elbow to right_wrist
#     for i in range(0,len(joints)):
#         for j in range(i+1,len(joints)):
#             body_lines.append(get_line(joint[i],joints[j]))
    # print("body_lines")
    # print(len(body_lines))
    # print(body_lines)
    return body_lines

def get_body_angles(body_lines):
    body_angles=[]
    for i in range(0,len(body_lines)):
        for j in range(i+1,len(body_lines)):
            # print("i,j ")
            # print(i)
            # print(j)
            numerator=np.dot(body_lines[i],body_lines[j])
            # print("numerator")
            # print(numerator)
            denominator=magnitude(body_lines[i])*magnitude(body_lines[j])
            # print("denominator")
            # print(denominator)
            argument = max(-1.0, min(numerator / denominator, 1.0))
            body_angles.append(math.acos(argument))
            # print("No eerrrorr")
    return body_angles  

def calculate_features(image):
    output=[]
    # Checking if the image is empty or not
    if image is None:
        result = "Image is empty!!"
        return
    # image=image_resize(image)
    
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        results = pose.process(image)
        # print("results")
        # print(results)
        if not results.pose_landmarks:
            return
        try:
            landmarks = results.pose_landmarks.landmark
            # print("landmarks")
            # print(landmarks)
            output=[]
            joints=get_joints(landmarks)
            # print("joints")
            # print(joints)
#             output=output+joints
            # get_body_lines(joints)
            bodyLines=get_body_lines(joints)
            output=output+get_body_angles(bodyLines)
            # print("output")
            # print(output)
            return output
        except Exception as e:
            print("Error in calculateFetaure :")
            print(e)
            

In [4]:
dataset=[]
count=0
errors=0


# image_path = 'Trikon.png'
# img = cv2.imread(image_path)
# image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# entry=calculate_features(image)
# print("Entry")

video_folder = 'Train Videos/1Bhujangasana'
video_paths = [os.path.join(video_folder, filename) for filename in os.listdir(
    video_folder) if filename.endswith(('.mp4', '.avi', '.mov'))]

for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (fps)
    frame_rate = 15
    prev = 0
    # i=1
    # while(i>0):
    while (cap.isOpened()):
        time_elapsed = time.time() - prev
        success, img = cap.read()
        # i = i-1
        if not success:
            break
        # print("Inamgs")
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            # # Convert the image to RGB (MediaPipe requires RGB images)
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            entry=calculate_features(image)
            # print("Entry")
            # print(entry)
            if(entry is not None):
                entry.append("Bhujangasana")
                dataset.append(entry)
            else:
                errors=errors+1
        # print("dataset")
        # print(dataset)
    cap.release()
        

In [5]:
len(dataset)

4286

In [6]:
video_folder = 'Train Videos/1JanuSirasana'
video_paths = [os.path.join(video_folder, filename) for filename in os.listdir(
    video_folder) if filename.endswith(('.mp4', '.avi', '.mov'))]

for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (fps)
    frame_rate = 15
    prev = 0
    while (cap.isOpened()):
        time_elapsed = time.time() - prev
        success, img = cap.read()
        if not success:
            break
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            entry=calculate_features(image)
            if(entry is not None):
                entry.append("JanuSirasana")
                dataset.append(entry)
            else:
                errors=errors+1
    cap.release()
        

In [7]:
len(dataset)

9118

In [8]:
video_folder = 'Train Videos/1Padmasana'
video_paths = [os.path.join(video_folder, filename) for filename in os.listdir(
    video_folder) if filename.endswith(('.mp4', '.avi', '.mov'))]

for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (fps)
    frame_rate = 15
    prev = 0
    while (cap.isOpened()):
        time_elapsed = time.time() - prev
        success, img = cap.read()
        if not success:
            break
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            entry=calculate_features(image)
            if(entry is not None):
                entry.append("Padmasana")
                dataset.append(entry)
            else:
                errors=errors+1
    cap.release()
        

In [9]:
len(dataset)

24582

In [10]:
video_folder = 'Train Videos/1Savasana'
video_paths = [os.path.join(video_folder, filename) for filename in os.listdir(
    video_folder) if filename.endswith(('.mp4', '.avi', '.mov'))]

for video_path in video_paths:
    print("video")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (fps)
    frame_rate = 15
    prev = 0
    while (cap.isOpened()):
        time_elapsed = time.time() - prev
        success, img = cap.read()
        if not success:
            break
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            entry=calculate_features(image)
            if(entry is not None):
                entry.append("Savasana")
                dataset.append(entry)
            else:
                errors=errors+1
    cap.release()
        

video
video
video
video
video
video
video
video
video
video
video
video
video


In [11]:
len(dataset)

28870

In [12]:
video_folder = 'Train Videos/1Tadasana'
video_paths = [os.path.join(video_folder, filename) for filename in os.listdir(
    video_folder) if filename.endswith(('.mp4', '.avi', '.mov'))]

for video_path in video_paths:
    print("video")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (fps)
    frame_rate = 15
    prev = 0
    while (cap.isOpened()):
        time_elapsed = time.time() - prev
        success, img = cap.read()
        if not success:
            break
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            entry=calculate_features(image)
            if(entry is not None):
                entry.append("Tadasana")
                dataset.append(entry)
            else:
                errors=errors+1
    cap.release()
        

video
video
video
video
video
video
video
video
video
video
video


In [13]:
len(dataset)

37702

In [14]:
video_folder = 'Train Videos/1Trikonasana'
video_paths = [os.path.join(video_folder, filename) for filename in os.listdir(
    video_folder) if filename.endswith(('.mp4', '.avi', '.mov'))]

for video_path in video_paths:
    print("video")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (fps)
    frame_rate = 15
    prev = 0
    while (cap.isOpened()):
        time_elapsed = time.time() - prev
        success, img = cap.read()
        if not success:
            break
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            entry=calculate_features(image)
            if(entry is not None):
                entry.append("Trikonasana")
                dataset.append(entry)
            else:
                errors=errors+1
    cap.release()
        

video
video
video
video
video
video
video
video
video
video


In [15]:
len(dataset)

47968

In [18]:
dataset[2]
len(dataset[2])

92

In [22]:
import pandas as pd
columns=[]
for i in range(1,92):
    columns.append('feature_'+str(i))

columns.append('Asana')
print(len(columns))
print(columns)
df = pd.DataFrame(dataset, columns = columns)
df

92
['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_67', 'feature_68', 'feature_69', 'feature_70', 'feature_71', 'feature_72

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,Asana
0,0.390868,1.368406,1.773186,1.337798,1.365364,1.583557,1.556073,1.531410,1.610182,0.117786,...,1.747249,1.781250,1.743426,0.028802,0.082979,0.106053,0.057659,0.112570,0.163168,Bhujangasana
1,0.393909,1.357082,1.784511,1.336309,1.366456,1.600494,1.591551,1.517436,1.624157,0.105722,...,1.734884,1.783613,1.746562,0.012928,0.075594,0.121104,0.074840,0.111041,0.170728,Bhujangasana
2,0.393437,1.364788,1.776805,1.336911,1.362524,1.604834,1.609035,1.523693,1.617900,0.098094,...,1.728531,1.786630,1.725106,0.024790,0.097186,0.104487,0.072396,0.117713,0.171663,Bhujangasana
3,0.407532,1.352882,1.788711,1.338714,1.362156,1.626340,1.609481,1.509508,1.632084,0.088721,...,1.698816,1.778740,1.724819,0.022999,0.088984,0.100680,0.089692,0.121432,0.167373,Bhujangasana
4,0.404212,1.356007,1.785585,1.337193,1.360121,1.626101,1.608750,1.512352,1.629241,0.091705,...,1.707345,1.780621,1.725619,0.018927,0.090176,0.100594,0.084238,0.119315,0.167928,Bhujangasana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47963,0.274227,0.690765,2.450828,1.351496,1.693270,0.636971,1.441375,1.264546,1.877046,1.780485,...,0.464627,2.938465,2.859577,0.555778,2.857997,2.958754,2.869404,2.405080,0.464853,Trikonasana
47964,0.266871,0.691470,2.450123,1.406439,1.643803,0.649509,1.398698,1.282914,1.858679,1.779573,...,0.471934,2.927455,2.897556,0.547342,2.858430,2.973222,2.877196,2.432285,0.445386,Trikonasana
47965,0.274568,0.700919,2.440674,1.376443,1.673465,0.650214,1.451093,1.248608,1.892984,1.809968,...,0.465741,2.921598,2.917987,0.544147,2.848456,3.000582,2.890308,2.461229,0.429234,Trikonasana
47966,0.260313,0.656677,2.484915,1.439414,1.671517,0.671098,1.462100,1.276372,1.865220,1.740372,...,0.485441,2.925656,2.948364,0.605338,2.815265,3.051170,2.862544,2.468881,0.395414,Trikonasana


In [23]:
df.to_csv('TrainData.csv',index=False)
df.shape

(47968, 92)